In [143]:
# ! pip install beautifulsoup4
# ! pip install requests

Update:
+ Finish scrap the product listing page
+ Know where to find the SKU 
+ Need to scrap available switches and price of each product

In [163]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
# page_no = 1
# url_link = "https://mechanicalkeyboards.com/shop/index.php?pg="+str(page_no)+"&l=product_list&c=1&show=100"

# Shop listing page

In [3]:
def get_soup(url_link):
    return BeautifulSoup(requests.get(url_link).text, 'html.parser')


In [9]:
def get_number_of_page():
    soup = get_soup("https://mechanicalkeyboards.com/shop/index.php?pg=1&l=product_list&c=1&show=100")
    no_page = soup.select_one('div.blog-pagination span')
    return no_page.text.split()[-1]



In [10]:
PRODUCT_SOUP_TAG = "div.product-name a"
IMAGE_URL_TAG = 'div.imagebox li a img'

product_name_list = []
product_url_list = []
product_img_url_list = []

#Get the last page of website (APPLY FOR THIS URL ONLY)
#"https://mechanicalkeyboards.com/shop/index.php?pg=1&l=product_list&c=1&show=100"
no_page = int(get_number_of_page())

for i in range(1,no_page+1):
        url_link = "https://mechanicalkeyboards.com/shop/index.php?pg="+str(i)+"&l=product_list&c=1&show=100"
        product_list_soup = get_soup(url_link)
        product_soup = product_list_soup.select(PRODUCT_SOUP_TAG)
        image_link = product_list_soup.select(IMAGE_URL_TAG)
        for (name,url) in zip(product_soup, image_link):
                #Get element
                product_name = name.text
                product_url = name['href']
                full_image_url = "https://mechanicalkeyboards.com/shop/" + url['src']
                
                #Add to list
                product_name_list.append(product_name)
                product_url_list.append(product_url)
                product_img_url_list.append(full_image_url)



# Product Detail Page


In [ ]:
PRODUCT_NAME_TAG = 'div.product-detail h4'
PRODUCT_SKU_TAG = 'div.product-id span'
SWITCH_TAG = 'table.edition_switch_list tr td.switch'
PRICE_TAG = 'table.edition_switch_list tr td.price'

def get_tag_detail_one(url,tag):
    return get_soup(url).select_one(tag)

def get_tag_detail(url,tag):
    return get_soup(url).select(tag)

def get_switch_table_len(url):
    return len(get_tag_detail(url, 'table.edition_switch_list tr'))

In [161]:
product_name_detail = []
product_sku = []
#sample product detail page
sample_product_url = product_url_list[1]

product_name = get_tag_detail_one(sample_product_url, PRODUCT_NAME_TAG)
sku = get_tag_detail_one(sample_product_url, PRODUCT_SKU_TAG)

switch = get_tag_detail(sample_product_url, SWITCH_TAG )
price = get_tag_detail(sample_product_url, PRICE_TAG)

#remove div tag in switch 
for i in range(len(switch)):
    switch[i].find('div').decompose() 

In [162]:
print(f"Keyboard:{product_name.text}\nSKU:{sku.text}")
print("Switch selection:")
for i in range(len(switch)):
    print(f"{switch[i].text}:{price[i].text}")

Keyboard:Ducky One 2 Mini v2 RGB LED  60% Double Shot PBT Mechanical Keyboard
SKU:DKON2061ST-_USPDAZT_
Switch selection:
Cherry MX Black:$99.00
Cherry MX Brown:$99.00
Cherry MX Blue:$99.00
Cherry MX Red:$99.00
Cherry MX Silver:$99.00
Gateron Yellow:$99.00
Gateron Red:$99.00
Gateron Brown:$99.00
Kailh Speed Pro Burgundy:$99.00
Kailh Speed Copper:$99.00
Kailh BOX White:$99.00
Kailh BOX Brown:$99.00
Kailh BOX Red:$99.00
Kailh BOX Thick Jade:$104.00
Kailh BOX Silent Pink:$104.00
Cherry MX Silent Black:$104.00
Kailh BOX Navy:$104.00
Cherry MX Silent Red:$104.00
Kailh Polia:$104.00
Gateron Silent Brown:$104.00
TTC Gold Pink:$109.00
TTC Bluish White:$109.00
TTC Heart:$109.00


In [167]:
switch_list =[switch[i].text for i in range(len(switch))]
switch_price_list = [price[i].text for i in range(len(price))]


In [175]:
def create_product(name, sku, switch, switch_price):
    product = {"name":name, "sku":sku,"switch":switch,"switch_price":switch_price}
    return product

ducky1 = create_product(product_name.text, sku.text, switch_list, switch_price_list)




{"name": "Ducky One 2 Mini v2 RGB LED  60% Double Shot PBT Mechanical Keyboard", "sku": "DKON2061ST-_USPDAZT_", "switch": ["Cherry MX Black", "Cherry MX Brown", "Cherry MX Blue", "Cherry MX Red", "Cherry MX Silver", "Gateron Yellow", "Gateron Red", "Gateron Brown", "Kailh Speed Pro Burgundy", "Kailh Speed Copper", "Kailh BOX White", "Kailh BOX Brown", "Kailh BOX Red", "Kailh BOX Thick Jade", "Kailh BOX Silent Pink", "Cherry MX Silent Black", "Kailh BOX Navy", "Cherry MX Silent Red", "Kailh Polia", "Gateron Silent Brown", "TTC Gold Pink", "TTC Bluish White", "TTC Heart"], "switch_price": ["$99.00", "$99.00", "$99.00", "$99.00", "$99.00", "$99.00", "$99.00", "$99.00", "$99.00", "$99.00", "$99.00", "$99.00", "$99.00", "$104.00", "$104.00", "$104.00", "$104.00", "$104.00", "$104.00", "$104.00", "$109.00", "$109.00", "$109.00"]}
<class 'str'>
